**тематическое моделирование**

Евгений Борисов borisov.e@solarl.ru

In [1]:
# http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html

# разложение частотной матрицы [ слова x документы ]  
#
# получаем матрицу с описанием тем [ слова х темы ]   
# и матрицу вероятностей событий "тема описывает документ"  [ темы х документы ]
# 
# [ слова x документы ] = [ слова х темы ] * [ темы х документы ]
# 
# p(w|d) = p(w|t) * p(t|d)

In [2]:
import sys
import re
import gzip
import pandas as pd

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

In [4]:
pd.options.display.max_colwidth = 200  

In [5]:
n_features = 1000
n_components = 10

df = pd.read_pickle('../data/text/news.pkl.gz')
print('текстов:',len(df))
df.sample(4)

текстов: 3196


,text,tag
2843,"Представители телеканала «Пятница» заявили о подаче иска о защите репутации к 20 пользователям Facebook за их комментарии по поводу ведущей программы «Ревизорро» Елены Летучей, пишут «Ведомости».\...",incident
2486,Церемония вручения наград Европейской киноакадемии за лучшие фильмы прошедшего года обошлась без сюрпризов: все главные призы ушли одному фавориту. А лучший композитор года — россиянин!\n\nТриумфа...,culture
1154,Два россиянина отодвинули Мартена Фуркада на 3-е место в гонке преследования в Эстерсунде\n\n4 декабря 2016 в 16:27\n\nSPORT.TUT.BY\n\nФранцузский биатлонист Мартен Фуркад из-за двух промахов на з...,sport
21,"Контрактники могут получить льготы при поступлении в вузы... и внести в\nГосдуму проект федерального закона, направленного на обеспечение доступности\nвысшего образования для лиц, проходивших воен...",politics


In [6]:
data = df['text'].tolist()

In [7]:
with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
    stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )

print('количество стоп-слов:',len(stopwords))

sorted(stopwords)

количество стоп-слов: 151


['а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [8]:
%%time 

df['text_clean'] = df['text'].str.lower() # приведение в lowercase

# замена символов-разделителей (-,_) на пробел
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'_', ' ', s))

# замена цифр
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' ', s))

# делим строки на слова
df['text_clean'] = df['text_clean'].apply(lambda t: [ w.strip() for w in t.split() if len(w.strip())>2 ] )

# удаление лишних слов
df['text_clean'] = df['text_clean'].apply(lambda t:[w for w in t if w not in stopwords])

CPU times: user 581 ms, sys: 16.8 ms, total: 597 ms
Wall time: 598 ms


In [9]:
df[['text_clean']].sample(10)

,text_clean
2498,"[версии, efa, европейской, киноакадемии, фильм, тони, эрдманн, заслужил, звание, лучшего, европейского, фильма, года, пишет, газета, ссылкой, сайт, академии, фильм, трагикомедия, рассказывает, муж..."
790,"[белорусские, галоши, подарил, иван, ургант, дизайнеру, обуви, кристиану, лубутену, декабря, tut, студии, передачи, вечерний, ургант, декабря, побывал, известный, французский, дизайнер, модельер, ..."
2806,"[подробности, категория, софт, создано, лаборатория, касперского, собирается, подавать, претензию, стороны, корпорации, microsoft, нарушается, антимонопольное, законодательство, сообщает, рамблер,..."
2853,"[телеканал, пятница, входит, холдинг, газпром, медиа, сообщил, подал, иск, защите, репутации, десяткам, пользователей, социальной, сети, facebook, претензии, канала, связаны, обсуждением, социальн..."
1233,"[пострадавшему, аварии, никите, шалаеву, солигорска, собраны, деньги, реабилитацию, декабря, auto, tut, реабилитацию, никиты, шалаева, вместе, сестрой, пострадавшего, аварии, августе, собрано, евр..."
975,"[российский, рубль, укрепился, byn0, составил, белорусского, рубля, российских, евро, снизился, byn0, рубля, доллар, укрепился, byn0, рубля, белорусской, валютно, фондовой, бирже, декабря, прошли,..."
2039,"[алексей, емелин, разбавил, скучный, матч, своими, силовыми, приемами, александр, радулов, проиграл, единоборство, овертайме, владислав, каменев, набрал, очка, ахл, питтсбург, аризона, евгений, ма..."
1251,"[спасибо, идеи, некоторые, реализовали, начальник, угаи, письмах, водителей, пешеходов, ноября, татьяна, матвеева, tut, госавтоинспекция, региона, проводит, проект, открытый, диалог, жители, облас..."
1718,"[москва, декабря, тасс, международное, рейтинговое, агентство, standard, poor, понизило, долгосрочный, краткосрочный, кредитные, рейтинги, татфондбанка, иностранной, национальной, валютах, ссс, по..."
1521,"[депутат, завила, работает, лишь, благо, украины, нардеп, надежда, савченко, ответила, обвинения, своей, работе, россию, заявив, слухи, полная, чепуха, проект, кремля, любые, другие, выпады, котор..."


In [10]:
# sorted(set([ w for t in  df['text_clean'] for w in t ]))

In [11]:
# собираем слова в строку
df['text_clean'] = df['text_clean'].apply(lambda t: ' '.join(t) )

In [12]:
%xdel stopwords

In [13]:
df[ df['text_clean'].str.len()<1 ]

,text,tag,text_clean


In [14]:
df.sample(10)

,text,tag,text_clean
1916,"МОСКВА, 13 дек — РИА Новости. Президент США Барак Обама арендовал офис в главном здании Всемирного фонда дикой природы (WWF) в Вашингтоне, пишет газета Washington Post со ссылкой на источники, зна...",politics,москва дек риа новости президент сша барак обама арендовал офис главном здании всемирного фонда дикой природы wwf вашингтоне пишет газета washington post ссылкой источники знакомые ситуацией согла...
1915,"В администрации уходящего президента США подтвердили, что соглашение заключено и у Обамы будет офис\n\nВашингтон, , 08:44 — REGNUM Действующий президент США Барак Обама арендовал офисное помещение...",politics,администрации уходящего президента сша подтвердили соглашение заключено обамы офис вашингтон regnum действующий президент сша барак обама арендовал офисное помещение здании всемирного фонда дикой ...
420,Парламент объявил импичмент президенту Южной Кореи\n\nопубликовано: 9 декабря 2016 в 10:23\n\nобновлено: 9 декабря 2016 в 11:20\n\nMeduza / TUT.BY\n\nПарламент Южной Кореи проголосовал за импичмен...,politics,парламент объявил импичмент президенту южной кореи опубликовано декабря обновлено декабря meduza tut парламент южной кореи проголосовал импичмент президенту страны пак кын тайное голосование прошл...
2262,"На военной базе в районе южнокорейского города Ульсан произошел взрыв, в результате которого пострадали более 20 человек, сообщает «Ренхап».\n\nПо предварительным данным, взорвался боеприпас. Отме...",incident,военной базе районе южнокорейского города ульсан произошел взрыв результате которого пострадали человек сообщает ренхап предварительным данным взорвался боеприпас отмечается большинство пострадавш...
1448,"Мехико, 13 декабря. Глава правительства Венесуэлы Николас Мадуро по итогам переговоров с ведущими экономическими советниками принял решение закрыть границу с Колумбией на 72 часа. На такой шаг ему...",politics,мехико декабря глава правительства венесуэлы николас мадуро итогам переговоров ведущими экономическими советниками принял решение закрыть границу колумбией часа шаг пришлось пойти избежать контраб...
1396,"Социологический опрос, проведенный Левада-центром, показал, почему россияне доверяют российскому президенту Владимиру Путину. Об этом сообщает «Интерфакс».\n\nСогласно данным исследования, 28% опр...",politics,социологический опрос проведенный левада центром показал почему россияне доверяют российскому президенту владимиру путину сообщает интерфакс согласно данным исследования опрошенных считают успешно...
59,"Британские ученые выведут капусту, предотвращающую рак Британские ученые\nработают над выведением нового сорта капусты брокколи с повышенным\nсодержанием сульфорафана - вещества, способного выводи...",health,британские ученые выведут капусту предотвращающую рак британские ученые работают выведением нового сорта капусты брокколи повышенным содержанием сульфорафана вещества способного выводить организма...
2914,"Как рассказал британскому изданию AutoExpress глава американского подразделения Volkswagen Генрих Вебкен, компания всерьез подумывает зайти на территорию, на которой сейчас правят модели Ford F-15...",auto,рассказал британскому изданию autoexpress глава американского подразделения volkswagen генрих вебкен компания всерьез подумывает зайти территорию которой правят модели ford dodge ram концерном свя...
1014,Евросоюз осудил ноябрьские расстрелы осужденных в Беларуси\n\n30 ноября 2016 в 14:52\n\nTUT.BY\n\nЕвросоюз осуждает смертную казнь белорусов Ивана Кулеша и Сергея Хмелевского. Об их расстреле стал...,politics,евросоюз осудил ноябрьские расстрелы осужденных беларуси ноября tut евросоюз осуждает смертную казнь белорусов ивана кулеша сергея хмелевского расстреле стало известно днях соответствующее заявлен...
1788,"ПЕКИН (Рейтер) - Промышленное производство и розничные продажи в Китае в ноябре росли более быстрыми темпами, чем ожидалось, а увеличение инвестиций в основные фонды совпало с прогнозами, в очеред...",economics,пекин рейтер промышленное произ

---

In [15]:
def print_top_words(model, feature_names, n_top_words=7):
    for topic_idx, topic in enumerate(model.components_):
        message = "Тема %d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

---

In [16]:
# tf features 
tf_vectorizer = CountVectorizer( max_df=0.95, min_df=2, max_features=n_features )
tf = tf_vectorizer.fit_transform(df['text_clean'])
tf_feature_names = tf_vectorizer.get_feature_names()

In [17]:
# LDA - латентное размещение Дирихле
lda = LatentDirichletAllocation( n_components=n_components, max_iter=5, 
                                learning_method='online', learning_offset=50.,
                                random_state=0 ).fit(tf)
print('\nLDA:\n')
print_top_words(lda, tf_feature_names)


LDA:

Тема 0: это очень которые время просто лет нужно
Тема 1: президент сша трамп президента заявил декабря глава
Тема 2: рублей автомобиль модели автомобилей модель автомобиля авто
Тема 3: беларуси tut декабря установлена внимание версия браузер
Тема 4: народов коренных севера малочисленных коми проекта края
Тема 5: object савченко нефти нефть тгк индекс оао
Тема 6: года году это также компании декабря которые
Тема 7: декабря tut время дтп фото результате области
Тема 8: года также декабря россии тысяч сообщает данным
Тема 9: динамо лучший мяч мира место стал чемпионата


---

In [18]:
# tf-idf features 
tfidf_vectorizer = TfidfVectorizer( max_df=0.95, min_df=2, max_features=n_features)

tfidf = tfidf_vectorizer.fit_transform(df['text_clean'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [19]:
# NMF (Frobenius norm) - неотрицательное матричное разложение
nmf = NMF( n_components=n_components, random_state=1,alpha=.1, l1_ratio=.5 ).fit(tfidf)
print('\nNMF(Frobenius norm):\n')
print_top_words( nmf, tfidf_feature_names )


NMF(Frobenius norm):

Тема 0: это года которые году также декабря время
Тема 1: трамп сша трампа дональд президент избранный президента
Тема 2: дтп водитель результате области мвд декабря происшествия
Тема 3: flash adobe javascript player проигрывателя html5 браузер
Тема 4: савченко украины партии надежда заявила лидер действия
Тема 5: рублей млн млрд долларов тысяч года году
Тема 6: динамо чемпионата матче мира очков матча лиги
Тема 7: народов севера коренных малочисленных края фестиваль июля
Тема 8: россии путин президент президента заявил глава россия
Тема 9: алеппо города сирии жителей восточной сутки тысяч


---

In [20]:
# NMF (generalized Kullback-Leibler divergence)  
nmf = NMF( n_components=n_components, random_state=1, beta_loss='kullback-leibler', 
          solver='mu', max_iter=1000, alpha=.1, l1_ratio=.5 ).fit(tfidf)
print('\nNMF(generalized Kullback-Leibler divergence):\n')
print_top_words(nmf, tfidf_feature_names )


NMF(generalized Kullback-Leibler divergence):

Тема 0: это фото также года декабря которая который
Тема 1: сша президент трамп президента ранее трампа также
Тема 2: декабря результате области человек сообщает сообщили пресс
Тема 3: декабря tut установлена поддерживает версия фото старая
Тема 4: это сегодня украины словам лидер декабря является
Тема 5: рублей также млн рамках области пресс народов
Тема 6: стал мира место декабря который чемпионата года
Тема 7: ученые человека которые людей человек лет жизни
Тема 8: россии декабря заявил страны президент также словам
Тема 9: также сообщает ранее года компания который тысяч
